## MySQL 테이블 생성 및 데이터 입력 방법

In [1]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [3]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [4]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

## 테이블 생성

In [5]:
cur = conn.cursor()
sql_create_table = '''
    create table if not exists users ( 
        uid varchar(20) not null primary key, 
        pwd char(44) not null, 
        uname varchar(20) not null,
        email varchar(40), 
        reg_date datetime default current_timestamp, 
        is_deleted int default 0 
    );
'''

In [6]:
cur.execute(sql_create_table)

0

## 데이터 추가

In [7]:
cur = conn.cursor() 
sql_insert = "INSERT INTO users(uid, pwd, uname) VALUES('admin', '1234', '관리자');"
cur.execute(sql_insert)

1

In [10]:
sql_select = 'select * from users;'         # 아직까진 hideisql에서는 확인 못함, 현재 cache 메모리에만 저장되어 있음. 데이터를 디스크에 써야함 -> conn.commit()
cur.execute(sql_select)
row = cur.fetchone()

In [11]:
row

('admin', '1234', '관리자', None, datetime.datetime(2021, 8, 25, 13, 29, 49), 0)

In [13]:
# 데이터를 하드 디스크에 쓰게 하는 명령
conn.commit()

- 여러건의 데이터 추가

In [14]:
sql_multi_insert = """
        INSERT INTO users(uid, pwd, uname) VALUES
            ('eskim', '1234', '김은숙'), ('wjlee', '1234', '이우정');
"""
cur.execute(sql_multi_insert)
conn.commit()

- Placeholder

In [15]:
sql_insert_ph = "INSERT INTO users(uid, pwd, uname) VALUES(%s, '1234', %s)"
uid, uname = 'djy', '대조영'
cur.execute(sql_insert_ph, (uid, uname))
conn.commit()

- 여러개의 게이터를 placeholder를 이용해서 insert

In [16]:
users = (('gdhong', '홍길동'), ('jbplark', '박재범'))
cur.executemany(sql_insert_ph, users)
conn.commit()

In [17]:
# for 반복문을 이용해서 insert 하기 (강사가 추천하는 방법)
users = (('gdhong2', '홍길동'), ('jbplark2', '박재범'))
for user in users:
    cur.execute(sql_insert_ph, user)
conn.commit()

## 마무리

In [18]:
cur.close()
conn.close()